<a href="https://colab.research.google.com/github/Amanda-Torres/api_flask_ngrok/blob/main/api_flask_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalando bibliotecas

In [1]:
!pip install -Uq Flask
!pip install -Uq pyngrok
!pip install -Uq flask-cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.3 MB/s eta 0:00:00
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importando bibliotecas

In [3]:
from pyngrok import ngrok
from google.colab import userdata
from flask import Flask, jsonify, request
from flask_cors import CORS

### Configurações do Ngrok

In [10]:
authtoken = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(authtoken)

# Crie um túnel HTTP na porta 5000
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://6ad7-34-86-128-22.ngrok-free.app" -> "http://localhost:5000"


### Criando Funções

In [11]:
def listar_alunos(filepath):
    """
    Função para listar alunos que participaram do chat.

    Args:
        filepath (str): O caminho para o arquivo de texto contendo as mensagens do chat.

    Returns:
        list: Uma lista de alunos únicos que participaram do chat.
    """
    alunos = set()  # Usar um conjunto para garantir unicidade
    with open(filepath, 'r') as file:
        for line in file:
            if 'From ' in line and 'To Everyone:' in line:
                aluno = line.split('From ')[1].split(' To Everyone:')[0]
                alunos.add(aluno)
    return list(alunos)  # Converter de volta para lista antes de retornar

In [13]:
def listar_mensagens_de_aluno(filepath, aluno):
    """
    Função para listar todas as mensagens de um aluno específico.

    Args:
        filepath (str): O caminho para o arquivo de texto contendo as mensagens do chat.
        aluno (str): O nome do aluno cujas mensagens devem ser listadas.

    Returns:
        list: Uma lista de mensagens enviadas pelo aluno especificado.
    """
    mensagens_do_aluno = []
    with open(filepath, 'r') as file:
        for line in file:
            if 'From ' in line and 'To Everyone:' in line:
                participante = line.split('From ')[1].split(' To Everyone:')[0]
                if participante == aluno:
                    mensagem = next(file).strip()
                    mensagens_do_aluno.append(mensagem)
    return mensagens_do_aluno


In [12]:
def encontrar_alunos_por_resposta(filepath, palavra):
    """
    Função para encontrar todos os alunos que responderam uma chamada com uma determinada palavra.

    Args:
        filepath (str): O caminho para o arquivo de texto contendo as mensagens do chat.
        palavra (str): A palavra a ser buscada nas respostas dos alunos.

    Returns:
        list: Uma lista de alunos que mencionaram a palavra em suas respostas.
    """
    alunos_com_resposta = set()
    with open(filepath, 'r') as file:
        for line in file:
            if 'From ' in line and 'To Everyone:' in line:
                participante = line.split('From ')[1].split(' To Everyone:')[0]
                resposta = next(file).strip()
                if palavra in resposta:
                    alunos_com_resposta.add(participante)
    return list(alunos_com_resposta)

In [14]:
def aluno_rei_do_chat(filepath):
    """
    Função para encontrar o aluno que mais participou do diálogo no chat.

    Args:
        filepath (str): O caminho para o arquivo de texto contendo as mensagens do chat.

    Returns:
        dict: Um dicionário com o aluno que mais participou e o número de mensagens enviadas.
    """
    participant_activity = {}
    with open(filepath, 'r') as file:
        for line in file:
            if 'From ' in line and 'To Everyone:' in line:
                participant = line.split('From ')[1].split(' To Everyone:')[0]
                participant_activity[participant] = participant_activity.get(participant, 0) + 1

    if not participant_activity:
        return {"error": "Nenhuma mensagem encontrada no arquivo"}

    most_active_participant = max(participant_activity.items(), key=lambda x: x[1])

    return {
        "most_active_participant": most_active_participant[0],
        "message_count": most_active_participant[1]
    }

### Aplicação Flask

In [15]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
  return "hello world!"

@app.route('/listar_alunos')
def listar_alunos_route():
  alunos = listar_alunos("/content/chat.txt")
  return alunos

@app.route('/find_response/<palavra>')
def call_response_detector(palavra):
    alunos = encontrar_alunos_por_resposta('/content/chat.txt', palavra)
    return alunos

@app.route('/mensagens_do_aluno/<fulano>')
def listar_msg_aluno(fulano):
  falas = listar_mensagens_de_aluno("/content/chat.txt",fulano)
  return falas

@app.route('/rei_do_chat')
def listar_rei_do_chat():
  aluno = aluno_rei_do_chat("/content/chat.txt")
  return aluno

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:21:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:21:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:25:35] "OPTIONS /rei_do_chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:25:35] "GET /rei_do_chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:25:37] "OPTIONS /listar_alunos HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:25:37] "GET /listar_alunos HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:26:22] "OPTIONS /find_response/Batman HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jun/2024 23:26:22] "GET /find_response/Batman HTTP/1.1" 200 -
INFO:werkzeug:127